# S&P 500 Historical Ticker Dataset with Delisted Companies

This notebook constructs a sample portfolio of S&P 500 consituents, including a proportional sample of delisted constituents. The aim is to allow for financial modeling and backtesting using a sample portfolio that mitigates survivorship bias without requiring curated portfolio construction.

Constituent data is sourced from the dataset made available at: [`fja05680/SP500`](https://github.com/fja05680/SP500) and price data is fetched using the 'yfinance' library.

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from pathlib import Path
from time import sleep


### Load Historical S&P 500 Ticker Data

In [ ]:
# Load your dataset (update path as needed)
csv_path = Path("Data/S&P 500 Historical Components & Changes(12-10-2024.csv")
sp500_constituents = pd.read_csv(csv_path)
sp500_constituents['date'] = pd.to_datetime(sp500_constituents['date'])

### Filter Relevant Time Period and Extract Tickers

In [ ]:
# Set study period
start_date = pd.to_datetime("2002-12-31")
end_date = pd.to_datetime("2023-01-01")

# Filter dataset within date range
filtered = sp500_constituents[
    (sp500_constituents['date'] >= start_date) & 
    (sp500_constituents['date'] <= end_date)
].copy()

# Extract and deduplicate tickers
filtered['tickers'] = filtered['tickers'].apply(lambda x: str(x).split(','))
all_tickers = set(ticker.strip() for tickers in filtered['tickers'] for ticker in tickers)
print(f"Unique tickers between {start_date.date()} and {end_date.date()}: {len(all_tickers)}")


### Download Adjusted Price Data with yfinance

In [ ]:
def download_price_data(tickers, start_date, end_date, delay=0.5):
    price_data = {}
    failed = []
    for ticker in tickers:
        try:
            print(f"Fetching {ticker}")
            data = yf.download(ticker, start=start_date, end=end_date, progress=False)['Adj Close']
            if not data.empty:
                price_data[ticker] = data
            else:
                failed.append(ticker)
        except Exception as e:
            print(f"Failed for {ticker}: {e}")
            failed.append(ticker)
        sleep(delay)
    return pd.DataFrame(price_data), failed


In [ ]:
prices_df, failed_tickers = download_price_data(all_tickers, start_date, end_date)

# Save optional CSVs
prices_df.to_csv("Data/sp500_adjusted_prices.csv")
pd.Series(failed_tickers).to_csv("Data/failed_tickers.csv", index=False)


### Summary of Downloaded Data

In [ ]:
print(f"Total successful tickers: {prices_df.shape[1]}")
print(f"Total failed tickers: {len(failed_tickers)}")
print("Sample of data:")
display(prices_df.iloc[:, :5].head())